# Modelo predictivo de factores relacionados para el reingreso de pacientes al area hospitalaria
- La finalidad del proyecto es conocer los factores que impactan directamente a la probabilidad de que un paciente reingrese al area hospitalaria una vez que este mismo fue dado de alta.
-

### 1. Carga y lectura de datos
- El data set que utilizare viene directamente de kaggle en el cual presenta las siguientes caracteristicas (features)
#### Features:
- Encounter ID (ID del encuentro)
- Patient number = (Número de paciente)
- Race = (Raza)
- Gender = (Género)
- Age = (Edad)
- Weight = (Peso)
- Admission type = (Tipo de admisión)
- Discharge disposition = (Destino al alta)
- Admission source = (Fuente de admisión)
- Time in hospital = (Tiempo en hospital)
- Payer code (Código = de pagador)
- Medical specialty = (Especialidad médica)
- Number of lab procedures = (Número de procedimientos de laboratorio)
- Number of procedures = (Número de procedimientos)
- Number of medications = (Número de medicamentos)
- Number of outpatient visits = (Número de visitas ambulatorias)
- Number of emergency visits = (Número de visitas de emergencia)
- Number of inpatient visits = (Número de visitas hospitalarias)
- Diagnosis 1 = (Diagnóstico 1)
- Diagnosis 2 = (Diagnóstico 2)
- Diagnosis 3 = (Diagnóstico 3)
- Number of diagnoses = (Número de diagnósticos)
- Glucose serum test result = (Resultado de prueba de glucosa en suero)
- A1c test result = (Resultado de prueba A1c)
- Change of medications = (Cambio de medicamentos)
- Diabetes medications = (Medicamentos para diabetes)

- 24 features for medications (24 características para medicamentos):

Específicamente: metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, y metformin-pioglitazone.
- readmitted = Variable objetivo:
- - NO: Si el paciente no fue readmitdo
- - >30: Si el paciente fue readmitido despues de 30 dias.
- - < 30: Si el paciente fue readmitido en menos de 30 dias. Clasificaria como 'SI'

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("brandao/diabetes")

print("Path to dataset files:", path)

C:\Users\asjer\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\asjer\.cache\kagglehub\datasets\brandao\diabetes\versions\1


In [3]:
import pandas as pd

data = pd.read_csv('../data/raw/diabetic_data.csv')

data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [8]:
data['readmitted']

0          NO
1         >30
2          NO
3          NO
4          NO
         ... 
101761    >30
101762     NO
101763     NO
101764     NO
101765     NO
Name: readmitted, Length: 101766, dtype: object

### Limpieza de datos

In [4]:
# Primero eliminaré las filas identificatorias, en este caso son las siguientes: encounter_id, patient_nbr

import numpy as np
data.drop(['patient_nbr', 'encounter_id'], axis=1, inplace=True)
data.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),?,1,1,7,3,?,?,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),?,1,1,7,2,?,?,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),?,1,1,7,1,?,?,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
# Ahora detecto que el data set tiene nulos en forma de ' ? ', creare un arreglo para eliminarlos, ya que son nulos
data.isnull().sum()

race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [6]:
# Para ello utilizaré NumPy
import numpy as np

data.replace('?', np.nan, inplace=True)

data.isnull().sum()

race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

Antes de eliminar los valores nulos, primero analizare su importancia para valorar si relleno los nulos con su media
race = Raza, esta columna podemos rellenar con el valor


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 48 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      99493 non-null   object
 1   gender                    101766 non-null  object
 2   age                       101766 non-null  object
 3   weight                    3197 non-null    object
 4   admission_type_id         101766 non-null  int64 
 5   discharge_disposition_id  101766 non-null  int64 
 6   admission_source_id       101766 non-null  int64 
 7   time_in_hospital          101766 non-null  int64 
 8   payer_code                61510 non-null   object
 9   medical_specialty         51817 non-null   object
 10  num_lab_procedures        101766 non-null  int64 
 11  num_procedures            101766 non-null  int64 
 12  num_medications           101766 non-null  int64 
 13  number_outpatient         101766 non-null  int64 
 14  numb